# MATH 300: Numerical Analysis I Recitation

## Instructor: Liam Doherty

## Meeting Time/Place: F 11:00AM, Curtis 453

## Office Hours: MRC Hours MR 5p-7p, W 12p-2p, or in Korman 282 by appointment

# Problem 4.4.22(a)

The equation $\newline \newline$
$$
\int_{0}^{x}\frac{1}{\sqrt{2\pi}}e^{-t^{2}/2}dt = 0.45
\newline
$$
can be solved for $x$ by using Newton's method with $\newline \newline$
$$
f(x) = \int_{0}^{x}\frac{1}{\sqrt{2\pi}}e^{-t^{2}/2}dt - 0.45
\newline
$$
and $\newline \newline$
$$
f'(x) = \frac{1}{\sqrt{2\pi}}e^{-x^{2}/2}.
\newline
$$
To evaluate $f$ at the approximation $p_{k}$, we need a quadrature formula to approximate $\newline \newline$
$$
\int_{0}^{p_{k}}\frac{1}{\sqrt{2\pi}}e^{-t^{2}/2}dt.
\newline
$$
Find a solution to $f(x) = 0$ accurate to within $10^{-5}$ using Newton's method with $p_{0} = 0.5$ and the Composite Simpson's rule.

## Solution: 

A quick computation reveals that $x < 2$ (there is underlying intuition here about what the $Z$-score for a $90\%$ confidence interval is):

In [ ]:
using QuadGK

f(x) = 1/√(2*pi)*exp(-x^2/2)
a = 0; b = 2
quadgk(f, a, b)

We can use $x = 2$ as an upper bound for our $b - a$ portion of our error term then, so that taking a step size of $h < 0.16559$ with Composite Simpson will be good enough so that our error does not get dominated by evaluation of the integral. (To fully justify the fact that we never get $p_{k} > 2$ for any $k$, we make an argument about the concavity: Notice that $\newline \newline$
$$
f''(x) = \frac{-x}{\sqrt{2\pi}}e^{-\frac{1}{2}x^{2}} < 0
\newline
$$
whenever $x > 0$; this means that the tangent lines will all lie above the graph. Combining this with the fact that $f(p_{0}) < 0$ (and perhaps also $f'(x) > 0$), we can argue that Newton's method does not overshoot the root.) Now, all we have to do is apply Newton's method, which is $\newline \newline$
$$
p_{k + 1} = p_{k} - \frac{f(p_{k})}{f'(p_{k})},
\newline
$$
or, writing it with our functions, $\newline \newline$
$$
p_{k + 1} = p_{k} - \frac{\int_{0}^{p_{k}}\frac{1}{\sqrt{2\pi}}e^{-x^{2}/2}dx - 0.45}{\frac{1}{\sqrt{2\pi}}e^{-\frac{p_{k}^{2}}{2}}}.
\newline
$$
This is going to be extraordinarily tedious to calculate by hand many times, so we will do one iteration in gory detail and the rest we will take for granted. For the integration, we use $h = 0.05$ and let the integrand of the integral term be denoted by $g$: $\newline \newline$
$$
\begin{align*}
    p_{1} &= 0.5 - \frac{\int_{0}^{0.5}\frac{1}{\sqrt{2\pi}}e^{-x^{2}/2}dx - 0.45}{\frac{1}{\sqrt{2\pi}}e^{-(0.5)^{2}/2}} \\
    &\approx 0.5 - \frac{\frac{0.05}{3}(g(0) + 4g(0.05) + 2g(0.1) + 4g(0.15) + 2g(0.2) + 4g(0.25) + 2g(0.3) + 4g(0.35) + 2g(0.4) + 4g(0.45) + g(0.5)) - 0.45}{\frac{1}{\sqrt{2\pi}}e^{-(0.5)^{2}/2}} \\
    &\approx 1.23435.
\end{align*}
\newline
$$
Now, for the next iteration, we would need to adapt the step size; Simpson's method requires a uniform mesh, so we would need to pick a step size that partitions $[0, 1.23435]$ into an integer number of subintervals. This could be accomplished by splitting the interval into $11$ subintervals each with length $0.123435$. However, this would make the computation fairly expensive to do by hand. We can obtain a few more iterates with code:

In [ ]:
function simpson(f::Function, a::Real, b::Real, n::Int)
    @assert a < b "Invalid interval!"
    h = (b - a)/n
    interior_nodes = [a + i*h for i in 1:n-1]
    
    # Start the sum with left endpoint
    sum = f(a)
    
    # Add in all interior nodes, accounting for even or odd index
    for (j, node) in enumerate(interior_nodes)
        if mod(j, 2) == 1
            sum += 4*f(node)
        elseif mod(j, 2) == 0
            sum += 2*f(node)
        end
    end
    
    # Add in right endpoint and multiply by h/3
    sum += f(b)
    sum *= h/3
    return sum
end

In [ ]:
function newton(f::Function, df::Function, p₀::Real; abs_tol = 1e-5, max_iters = 100, verbose = true)
    converged = false
    n = 0
    
    p = p₀
    while !converged
        n += 1
        p_old = p
        
        @assert df(p) != 0 "The derivative evaluated at the current point is zero! Cannot proceed. Last point: $(p)"
        
        # Main step for algorithm
        p = p - f(p)/df(p)
        
        # Status updates
        if verbose
            println("Old p: $(p_old), New p: $(p), Iteration: $(n)")
        end
        
        # Check convergence
        if abs(p - p_old) < abs_tol
            converged = true
            return p
        end
        
        # Return if algorithm does not converge
        if n == max_iters
            println("Did not converge in $(max_iters) iterations! Returning last value of p.")
            return p
        end
    end
end

In [ ]:
g(x) = 1/√(2*pi)*exp(-x^2/2)
# Always using 10 subintervals for integration; this ensures that we will always be within 1e-5 in our approximation.
f(x) = simpson(g, 0, x, 10) - 0.45; df(x) = g(x)
newton(f, df, 0.5)

We only required 6 iterations to achieve the desired accuracy! Let's check that this agrees with a version of the code that uses packages to accomplish the integration and rootfinding tasks:

In [ ]:
using Roots
using QuadGK

g(x) = 1/√(2*pi)*exp(-x^2/2)
f(x) = quadgk(g, 0, x)[1] - 0.45; df(x) = g(x) # quadgk is NOT using Composite Simpson, but allows us to check answer

find_zero((f, df), 0.5, Roots.Newton())

# Composing Algorithms

In a similar vein to the last problem, we can compose other algorithms that we have worked with together to solve problems. Consider some data generated by $\sin(x)$:

In [ ]:
xvals = collect(0.:1.:2*pi)
yvals = sin.(xvals)
data = collect(zip(xvals, yvals))

The goal is to approximate the root of $\sin(x)$ in $(0, 2\pi)$ (notice the lack of inclusion). Given data of this form, we can construct an interpolant, using `DataInterpolations.jl`:

In [ ]:
using DataInterpolations
interpolant = LagrangeInterpolation(yvals, xvals)

We can evaluate the interpolant away from the nodes:

In [ ]:
println(interpolant(pi)) # Expect that this is near zero

In [ ]:
using Plots
plot(interpolant, xlabel = "x", ylabel = "y", label = "Interpolant")
scatter!(xvals, sin.(xvals), label = "Data")

Now, we can use an algorithm for rootfinding **on the interpolant** to approximate the root of the function which generated the original data. We can do that using `Roots.jl` with our interpolant constructed from `DataInterpolations.jl`:

In [ ]:
using Roots
root = find_zero(interpolant, (xvals[2], xvals[end - 1])) # Don't include boundary points where roots are

We can check the error from the true solution (which is of course $\pi$):

In [ ]:
abs(root - pi)

Not bad! Adding more points makes it even better:

In [ ]:
using DataInterpolations, Roots
xvals = collect(0.:0.5:2*pi)
yvals = sin.(xvals)
interpolant = LagrangeInterpolation(yvals, xvals)
root = find_zero(interpolant, (xvals[2], xvals[end - 1]))
abs(root - pi)

So, we see that given data, we can also do rootfinding by first constructing an interpolant of the data (also called a **surrogate**) and then doing rootfinding with that function.

I also want to demonstrate **Runge's Phenomenon**, which is something that can happen when doing Lagrange interpolation on certain types of data. You would expect that using more data points would lead to more accurate approximations, but surprisingly, this is not always the case:

In [ ]:
using Plots
f(x) = 1/(1 + 25*x^2)
xvals = collect(-1:0.5:1)
yvals = f.(xvals)
interpolant = LagrangeInterpolation(yvals, xvals)
plot(interpolant, xlabel = "x", label = "Interpolant", legend = :outertopright,
    title = "Witch of Agnesi: Runge's Phenomenon")
scatter!(xvals, yvals, label = "Data")

fine_mesh = LinRange(-1, 1, 101)
plot!(fine_mesh, f.(fine_mesh), label = "Truth")

This does not look immediately alarming, but what if we increased the number of points in the dataset?

In [ ]:
f(x) = 1/(1 + 25*x^2)
xvals = collect(-1:0.1:1)
yvals = f.(xvals)
interpolant = LagrangeInterpolation(yvals, xvals)
plot(interpolant, xlabel = "x", label = "Interpolant", legend = :outertopright,
    title = "Witch of Agnesi: Runge's Phenomenon")
scatter!(xvals, yvals, label = "Data")

fine_mesh = LinRange(-1, 1, 101)
plot!(fine_mesh, f.(fine_mesh), label = "Truth")

Now, the interpolant is behaving wildly near the ends of the interval. And it gets even worse:

In [ ]:
f(x) = 1/(1 + 25*x^2)
xvals = collect(-1:0.05:1)
yvals = f.(xvals)
interpolant = LagrangeInterpolation(yvals, xvals)
plot(interpolant, xlabel = "x", label = "Interpolant", legend = :outertopright,
    title = "Witch of Agnesi: Runge's Phenomenon")
scatter!(xvals, yvals, label = "Data")

fine_mesh = LinRange(-1, 1, 101)
plot!(fine_mesh, f.(fine_mesh), label = "Truth")

You may ask: This looks bad! Does the error bound tell us that it could be this bad? The answer: Yes, of course it does! Recall the form of the error: $\newline \newline$
$$
f(x) - P(x) = \frac{f^{(n + 1)}(\xi)}{(n + 1)!}\prod_{k = 0}^{n}(x - x_{k}).
\newline
$$
Really, if we are evaluating at a point, the thing that can go wrong is that the derivative might be excessively large.

In [ ]:
using ForwardDiff
df(x) = ForwardDiff.derivative(f, x)
df(1 - 1e-3)

In [ ]:
# using TaylorSeries
# Taylor expand f about the point 1
t = taylor_expand(f, 1.; order = 100)
println(t)
t(0.) ≈ f(1.)

In [ ]:
# Calculate a derivative of the Taylor expansion at 1 - 1e-3
println(differentiate(t, 1)(-1e-3))
differentiate(t, 1)(-1e-3) ≈ df(1 - 1e-3)

In [ ]:
# Print some (scaled) maximums of derivatives of the function
xvals = collect(-1:0.01:1)
for n = 1:50
    println(maximum(differentiate(t, n + 1).(xvals))/factorial(big(n + 1)))
end

We see that the term $\newline \newline$
$$
\frac{1}{(n + 1)!}\max_{\xi \in (-1, 1)}|f^{n + 1}(\xi)|
\newline
$$
is blowing up as $n$ is getting larger. Thus, the error bounds are containing the information about Runge's Phenomenon.